# Task

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import doctest
import implicit
from implicit.als import AlternatingLeastSquares
import sklearn
from sklearn.model_selection import train_test_split
import scipy
from scipy.sparse import csr_matrix
import common2
import os

In [3]:
os.environ['MKL_NUM_THREADS'] = '1'

In [4]:
MOCK = False

In [5]:
if not MOCK:
    pd_data = pd.read_csv("./data/ratings.csv")
else:
    pd_data = pd.read_csv("./data/ratings_mock.csv")

In [6]:
pd_data = pd_data.sort_values(['userId', 'timestamp'])

# Algorithm

In [7]:
data = pd_data

In [8]:
train, test = common2.split_train_test(data)

In [ ]:
user_item = common2.to_sparce_matrix(train)

8002758


In [ ]:
model = AlternatingLeastSquares()

In [ ]:
model.fit(user_item.T.tocsr())

100%|██████████| 15.0/15 [02:08<00:00,  6.24s/it]


In [ ]:
U_train, I_train = user_item.shape

Чтобы избежать работы на холодном старта. Когда нет никакой информации о пользователе и следовательно они все равны. Было принято решение, для каждого из пользователей в тестовой группе знать случайных 80 процентов просмотренных фильмов с высокой оценкой, и по ним попытаться предсказать, фильмы, которые они уже посмотрели, но не входящие в эти 80 процентов.

In [ ]:
raw_information_about_test = []
for index in test['userId'].unique():
    values = test[test['userId'] == index]
    first, second = train_test_split(values, train_size=0.8)
    raw_information_about_test.append(first)

In [ ]:
information_about_test = pd.concat(raw_information_about_test, ignore_index=True)

In [ ]:
test_user_item = common2.to_sparce_matrix(information_about_test)

In [ ]:
list_of_approx = []
list_of_label = []

In [ ]:
k_top = 10

In [ ]:
for u in test['userId'].unique():
    list_of_approx.append(model.recommend(u, test_user_item, k_top))
    list_of_label.append(test[test['userId'] == u]['movieId'].values)

In [ ]:
def mean_ndgx(list_of_label, list_of_approx,  k):
    assert (len(list_of_label) == len(list_of_approx))
    return sum(common2.ndcg(label, approx, k) \
               for label, approx in zip(list_of_label, list_of_approx)) / len(list_of_label)

In [ ]:
mean_ndgx(list_of_label, list_of_approx, k_top)